# Aircraft-Collaborative Decision Making (ACDM) Performance Analysis
Jupyter Notebook for Aircraft-Collaborative Decision Making (ACDM) Analysis - Simple Data Viz

## Preface
This is just a simple data visualisation notebook using airport flight datasets - for ACDM performance measurement purposes.    
According to [Skytrax](https://www.worldairlineawards.com/worlds-most-improved-airlines-2019/), Philippine Airlines have been named the World's Most Improved Airline in 2019.      
Hence for the performance analysis of this excercise, we will be narrowing in on Philippine Airlines and make **simple data viz** of its performance, and observing its improvements.       
<br/>
## Some important information of Philippine Airlines:      
**IATA Code: PR**      
**ICAO Code: PAL**

### Make sure Jupyter is in correct directory
Check current directory with 'pwd'.
```
pwd 
```

If not in correct directory, 'cd' to the right folder. eg.
```
cd Analysis/
```


In [ ]:
pwd

In [ ]:
cd Analysis/

In [ ]:
pwd

# Setup Pandas & DataFrame

In [ ]:
import pandas as pd 


# Key in your file name
#excelFileName = 'sample_short_1.xlsx'
#sheetName = 'Sheet1'
excelFileName = 'sampleMarApr.xlsx'
sheetName = 'MarApr2018'

# Create DataFrame
df = pd.read_excel(excelFileName, sheet_name = sheetName)
#df = pd.read_excel(excelFileName, sheet_name = sheetName).astype(str)  # If there is a need to convert to string as the long int will automatically be converted down


#print(df.head())

#print(df.shape)

print(df.columns.values)

# Preparing Datasets
## 1. Filter to One Airline 
Philippine Airlines for our case

## 2. Drop useless datasets

A. For our given dataset, we should remove the columns that we dont need to minimise size of dataframe.     
we only need AOBT and EOBT           

B. we have to remove datasets that is Null for any attribute    
(those with Null is useless to us as we cannot do our desired operation of AOBT-EOBT)

## 3. Convert Date-Time of AOBT and EOBT to seconds
This is for easier operation and handling of the value given

## 4. Order the datasets chronologically
For our given dataset, we have to order them chronologically by Date-Time

## 5. Create another csv/xlsx file 
(optional, you can also work with current dataframe)



In [ ]:
##################################
#### 1. Filter to One Airline ####
##################################
# Filter to remaining datasets with Philippines Airlines



#df = df.dropna()
#print(df)
def filterIATA(df, IATAcode):
  return df.loc[df['IATA'] == 'PR']

df = filterIATA(df, 'PR')
df = df.reset_index(drop=True)

#print(df.loc[df['IATA'] == 'PR'])
print(df)



In [ ]:
##################################
#### 2. Drop Useless Datasets ####
##################################



# A. Streamline dataset (only need AOBT and EOBT)
#################################################
df = df[['IATA aircraft','ICAO aircraft','Arrival/Departure','IATA','ICAO','EOBT',
 'AOBT','CALLSIGN']]




# B. Remove datasets with Null/NaN in any attribute
###################################################

#Note that the blank cells given are not really a np.NaN object. 
#its a null, which pandas does not recognise and cant use df.dropna()
#we need to replace the null with np.nan first.
'''
OR
filter = df["Tenant"] != ""
dfNew = df[filter]
'''
df['EOBT'].replace('', np.nan, inplace=True)
df['AOBT'].replace('', np.nan, inplace=True)

df = df.dropna()
df = df.reset_index(drop=True) #reset index back to order( 0,1,2,3...)
print(df)


In [ ]:
##########################################################
#### 3. Convert Date-Time of AOBT and EOBT to seconds ####
##########################################################




#retrieve min and max values first - to be used later on - to be used as references
minEOBT = str(df['EOBT'].min())
maxEOBT = str(df['EOBT'].max())
minEOBT = removeFloat(minEOBT)
maxEOBT = removeFloat(maxEOBT)
print(minEOBT)
print(maxEOBT)


# Current data type is in float.
# Note Not sure why converting float to int turns '2.018033e+13' to '-2147483648 '
# Approach: Convert to string then drop the floating point
###########################################################
print(df)
def removeFloat(x):
  #return x[:,-2]
  return x[:-2]

df[["AOBT", "EOBT"]] = df[["AOBT", "EOBT"]].astype(str)



#df[["AOBT", "EOBT"]] = df.loc["AOBT", "EOBT"].apply(removeFloat)
#df = df.apply(lambda x: removeFloat(x) if x.name in ['EOBT', 'AOBT'] else x, axis=1)
df['AOBT'] = df['AOBT'].apply(removeFloat)
df['EOBT'] = df['EOBT'].apply(removeFloat)

print(df)





# Convert to seconds -- deducted with base reference (we use minEOBT as base reference)
##########################################################
from datetime import datetime

# input: string of yyyymmddhhmmss | output: datetime
def createDateTime(datetimeStr):
  year = int(datetimeStr[0:4])
  month = int(datetimeStr[4:6])
  day = int(datetimeStr[6:8])
  hour = int(datetimeStr[8:10])
  minute = int(datetimeStr[10:12])
  second = int(datetimeStr[12:14])
  return datetime(year=year,
                month=month,
                day=day,
                hour=hour,
                minute=minute,
                second=second)

# We use the min EOBT as base reference
minRefDateTime = createDateTime(minEOBT)

# Convert to seconds (relative) -- deducted with base reference too 
def secondsRelativeConvert(dateTimeStr):
  thisDateTime = createDateTime(dateTimeStr)
  timeDelta = thisDateTime - minRefDateTime
  timeDeltaSeconds = timeDelta.total_seconds()
  return int(timeDeltaSeconds)


#___________________________________________________________________________________________________
### Now we are ready to use secondsConvert() to convert the string AOBT & EOBT to seconds (relative)
# Note they are now in Integer form.
df['AOBT'] = df['AOBT'].apply(secondsRelativeConvert)
df['EOBT'] = df['EOBT'].apply(secondsRelativeConvert)

print(df)


In [ ]:
##########################################################
#### 4. Order the datasets chronologically ####
##########################################################
# From top to bottom, from  earliest to latest.
# we take EOBT as a gauge. ---note in long run, we should use a better indicator, but this works for now as the airlines is not flying so often that simultaneously flight happens.

df.sort_values(by=['EOBT'], inplace=True, ascending=True)
df = df.reset_index(drop=True)
print(df)


# Further Calculations
## Generate AOBT - EOBT data
Now with AOBT and EOBT expressed in seconds (integers),       
we can find their difference to get AOBT-EOBT in seconds(integers) as well.     

This is so we can have **3 plots on the same graph**:
- AOBT
- EOBT
- AOBT-EOBT
And visualise the trends/dynamics between these 3 variables.

In [ ]:
############################
#### Generate AOBT-EOBT ####
############################
df['AOBT-EOBT'] = df['AOBT'] - df['EOBT']
print(df)

In [ ]:
# Generate new xslx file from final DataFrame

In [ ]:
#################################################################
#### (Optional) Generate new xslx file from final DataFrame #####
#################################################################

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

newFileName = 'sampleMarAprNEW.xlsx'
newSheetName = 'Sheet1'

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(newFileName, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name=newSheetName)

# Close the Pandas Excel writer and output the Excel file.
writer.save()


In [ ]:
# Data Visualisation
## With M

In [ ]:
# Import libraries
import matplotlib.pyplot as plt 
import numpy as np 

In [ ]:
print(int('28'))